In [1]:
# MLP with automatic validation set
import numpy as np # import numpy library
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql.functions import mean, min, max
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer
from matplotlib.pyplot import figure
from datetime import datetime

%matplotlib inline

pd.set_option("display.max_columns", None)

spark = SparkSession \
       .builder \
       .appName("Rain Forecase") \
       .config("spark.executor.heartbeatInterval","60s")\
       .getOrCreate()

rdd = spark.read.format("csv").option("header", "true").load("/Users/rajeevkumar/Downloads/Coursera/rainfall in india 1901-2015.csv")
rdd.createOrReplaceTempView("rainfall")
annualrain = spark.sql("select *  from rainfall")
annualrain.show()

+--------------------+----+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+
|         SUBDIVISION|YEAR|   JAN|   FEB|   MAR|   APR|   MAY|   JUN|   JUL|   AUG|   SEP|   OCT|   NOV|   DEC| ANNUAL|Jan-Feb|Mar-May|Jun-Sep|Oct-Dec|
+--------------------+----+------+------+------+------+------+------+------+------+------+------+------+------+-------+-------+-------+-------+-------+
|ANDAMAN & NICOBAR...|1901| 49.20| 87.10| 29.20|  2.30|528.80|517.50|365.10|481.10|332.60|388.50|558.20| 33.60|3373.20| 136.30| 560.30|1696.30| 980.30|
|ANDAMAN & NICOBAR...|1902|  0.00|159.80| 12.20|  0.00|446.10|537.10|228.90|753.70|666.20|197.20|359.00|160.50|3520.70| 159.80| 458.30|2185.90| 716.70|
|ANDAMAN & NICOBAR...|1903| 12.70|144.00|  0.00|  1.00|235.10|479.90|728.40|326.70|339.00|181.20|284.40|225.00|2957.40| 156.70| 236.10|1874.00| 690.60|
|ANDAMAN & NICOBAR...|1904|  9.40| 14.70|  0.00|202.40|304.50|495.10|502.00|160.10|820.4

In [2]:
annualrain.printSchema()

root
 |-- SUBDIVISION: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- JAN: string (nullable = true)
 |-- FEB: string (nullable = true)
 |-- MAR: string (nullable = true)
 |-- APR: string (nullable = true)
 |-- MAY: string (nullable = true)
 |-- JUN: string (nullable = true)
 |-- JUL: string (nullable = true)
 |-- AUG: string (nullable = true)
 |-- SEP: string (nullable = true)
 |-- OCT: string (nullable = true)
 |-- NOV: string (nullable = true)
 |-- DEC: string (nullable = true)
 |-- ANNUAL: string (nullable = true)
 |-- Jan-Feb: string (nullable = true)
 |-- Mar-May: string (nullable = true)
 |-- Jun-Sep: string (nullable = true)
 |-- Oct-Dec: string (nullable = true)



In [3]:
annualrain.describe().show()

+-------+--------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+
|summary|         SUBDIVISION|              YEAR|               JAN|               FEB|              MAR|              APR|               MAY|               JUN|               JUL|              AUG|               SEP|              OCT|              NOV|               DEC|            ANNUAL|           Jan-Feb|           Mar-May|           Jun-Sep|           Oct-Dec|
+-------+--------------------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+-----------

In [4]:
annualrain.describe('JAN').show()

+-------+------------------+
|summary|               JAN|
+-------+------------------+
|  count|              4116|
|   mean|18.957320038910485|
| stddev| 33.58537112934876|
|    min|              0.00|
|    max|                NA|
+-------+------------------+



In [5]:
len(annualrain.columns)

19

In [6]:
annualrain.count()

4116

In [7]:
annualrain.schema['JAN'].dataType

StringType

In [8]:
from pyspark.sql.functions import col , column

for x in annualrain.schema.names:
    if x != 'SUBDIVISION':
        if 'annualrain1' in locals():
            annualrain1 = annualrain1.withColumn(x, col(x).cast("float"))
        else:
            annualrain1 = annualrain.withColumn(x, col(x).cast("float"))
annualrain1.printSchema()

root
 |-- SUBDIVISION: string (nullable = true)
 |-- YEAR: float (nullable = true)
 |-- JAN: float (nullable = true)
 |-- FEB: float (nullable = true)
 |-- MAR: float (nullable = true)
 |-- APR: float (nullable = true)
 |-- MAY: float (nullable = true)
 |-- JUN: float (nullable = true)
 |-- JUL: float (nullable = true)
 |-- AUG: float (nullable = true)
 |-- SEP: float (nullable = true)
 |-- OCT: float (nullable = true)
 |-- NOV: float (nullable = true)
 |-- DEC: float (nullable = true)
 |-- ANNUAL: float (nullable = true)
 |-- Jan-Feb: float (nullable = true)
 |-- Mar-May: float (nullable = true)
 |-- Jun-Sep: float (nullable = true)
 |-- Oct-Dec: float (nullable = true)



In [9]:
annualrain1.stat.corr('JAN','FEB')

0.45554599439644017

In [10]:
from pydoc import help
from scipy.stats.stats import pearsonr

annualrain2 = annualrain1.toPandas()
pearsonr(annualrain2['JAN'],annualrain2['FEB'])

(nan, 1.0)

In [11]:
annualrain3 = annualrain2.dropna()
pearsonr(annualrain3['JAN'],annualrain3['FEB'])

(0.4564589, 1.0788232732108767e-209)

In [12]:
annualrain2.describe()

,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANNUAL,Jan-Feb,Mar-May,Jun-Sep,Oct-Dec
count,4116.000000,4112.000000,4113.000000,4110.000000,4112.000000,4113.000000,4111.000000,4109.000000,4112.000000,4110.000000,4109.000000,4105.000000,4106.000000,4090.000000,4110.000000,4107.000000,4106.000000,4103.000000
mean,1958.218628,18.957319,21.805326,27.359196,43.127434,85.745415,230.234451,347.214355,290.263489,197.361923,95.507011,39.866165,18.870581,1411.008911,40.747784,155.901749,1064.724854,154.100494
std,33.140896,33.585373,35.909489,46.959423,67.831169,123.234901,234.710754,269.539673,188.770477,135.408340,99.519135,68.685410,42.369610,903.846558,59.308277,201.316971,707.741516,166.942657
min,1901.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,62.299999,0.000000,0.000000,57.400002,0.000000
25%,1930.000000,0.600000,0.600000,1.000000,3.000000,8.600000,70.350006,175.600006,155.975006,100.525002,14.600000,0.700000,0.100000,804.500000,4.100000,24.049999,573.849976,34.200001
50%,1958.000000,6.000000,6.700000,7.800000,15.700000,36.599998,138.699997,284.799988,259.399994,173.899994,65.199997,9.500000,3.000000,1121.300049,19.200001,74.800003,881.099976,98.199997
75%,1987.000000,22.200001,26.799999,31.299999,49.950001,97.199997,305.150024,418.399994,377.799988,265.799988,148.399994,46.099998,17.500000,1644.775024,50.375000,196.949997,1288.174927,213.500000
max,2015.000000,583.700012,403.500000,605.599976,595.099976,1168.599976,1609.900024,2362.800049,1664.599976,1222.000000,948.299988,648.900024,617.500000,6331.100098,699.500000,1745.800049,4536.899902,1252.500000


In [13]:
annualrain2.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
YEAR,4116.0,1958.218628,33.140896,1901.000000,1930.000000,1958.000000,1987.000000,2015.000000
JAN,4112.0,18.957319,33.585373,0.000000,0.600000,6.000000,22.200001,583.700012
FEB,4113.0,21.805326,35.909489,0.000000,0.600000,6.700000,26.799999,403.500000
MAR,4110.0,27.359196,46.959423,0.000000,1.000000,7.800000,31.299999,605.599976
APR,4112.0,43.127434,67.831169,0.000000,3.000000,15.700000,49.950001,595.099976
MAY,4113.0,85.745415,123.234901,0.000000,8.600000,36.599998,97.199997,1168.599976
JUN,4111.0,230.234451,234.710754,0.400000,70.350006,138.699997,305.150024,1609.900024
JUL,4109.0,347.214355,269.539673,0.000000,175.600006,284.799988,418.399994,2362.800049
AUG,4112.0,290.263489,188.770477,0.000000,155.975006,259.399994,377.799988,1664.599976
SEP,4110.0,197.361923,135.408340,0.100000,100.525002,173.899994,265.799988,1222.000000


In [14]:
import pandas_profiling
annualrainPD = annualrain.toPandas()
annualrainPD = annualrainPD.dropna()
pandas_profiling.ProfileReport(annualrain2)

Number of variables,19
Number of observations,4116
Total Missing (%),0.1%
Total size in memory,321.6 KiB
Average record size in memory,80.0 B
Numeric,16
Categorical,1
Boolean,0
Date,0
Text (Unique),0
Rejected,2
